In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from fredapi import Fred
import requests
from datetime import datetime
import seaborn as sns
from functools import reduce

#### DATA COLLECTION

fred = Fred(api_key='d85eb85e4c7b9c4a6162731e87702c0c')
stock_price = yf.download('^GSPC' , start='1950-01-01',end='2024-11-28',interval="1mo")
stock_price = stock_price['Adj Close']
stock_price = stock_price.dropna(axis=0)

tickers = ['M2V','UNRATE','CPIAUCNS','PPIACO','FEDFUNDS','NFCI','DTWEXBGS']


ticker_data = {}
for ticker in tickers:
    
    data = {'Date': fred.get_series(ticker).index,
            ticker: fred.get_series(ticker).values
           }
    ticker_data[ticker + "_data"] = pd.DataFrame(data)
    
    
    

dataframes = [ticker_data['M2V_data'], ticker_data['UNRATE_data'],ticker_data['CPIAUCNS_data'],ticker_data['PPIACO_data'],ticker_data['FEDFUNDS_data'],stock_price]

data = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), dataframes)



data = data.sort_values(by='Date',ascending=True,ignore_index=True)
data = data[data['Date']>'1984-12-30'].sort_values(by='Date',ascending=True,ignore_index=True)

data.to_csv('Initial_data.csv',index=False)

In [67]:
df = pd.read_csv('Initial_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,M2V,UNRATE,CPIAUCNS,PPIACO,FEDFUNDS,Adj Close
0,1985-01-01,1.799,7.3,105.5,103.4,8.35,179.630005
1,1985-02-01,NaN,7.2,106.0,103.3,8.50,181.179993
2,1985-03-01,NaN,7.2,106.4,103.1,8.58,180.660004
3,1985-04-01,1.795,7.3,106.9,103.3,8.27,179.830002
4,1985-05-01,NaN,7.2,107.3,103.5,7.97,189.550003


In [82]:
df['M2V'] = df['M2V'].fillna(method='ffill')
df.tail()

C:\Users\ASUS ROG\AppData\Local\Temp\ipykernel_15084\1191620724.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['M2V'] = df['M2V'].fillna(method='ffill')


,Date,M2V,UNRATE,CPIAUCNS,PPIACO,FEDFUNDS,Adj Close
474,2024-07-01,1.389,4.3,314.540,257.326,5.33,5522.299805
475,2024-08-01,1.389,4.2,314.796,255.394,5.33,5648.399902
476,2024-09-01,1.389,4.1,315.301,252.737,5.13,5762.479980
477,2024-10-01,1.389,4.1,315.664,253.452,4.83,5705.450195
478,2024-11-01,1.389,4.2,NaN,NaN,4.64,6032.379883


In [89]:
oil = pd.read_csv("F:/Okanagan College/3rd Semester/DSCI 401/Project/DCOILWTICO.csv")
oil['DATE'] = pd.to_datetime(oil['DATE'])
oil.head()

,DATE,DCOILWTICO
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87


In [90]:
filtered_df = oil[oil['DATE'].dt.day==1].reset_index(drop=True)
filtered_df['DATE'] = pd.to_datetime(filtered_df['DATE'])

In [91]:
filtered_df.rename(columns={'DATE': 'Date'}, inplace=True)

In [98]:
from functools import reduce
dataframes = [df,filtered_df]
data = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), dataframes)
data['DCOILWTICO'] = data['DCOILWTICO'].fillna(method='ffill')

C:\Users\ASUS ROG\AppData\Local\Temp\ipykernel_15084\2824773197.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['DCOILWTICO'] = data['DCOILWTICO'].fillna(method='ffill')


In [177]:
gold = pd.read_excel("F:/Okanagan College/3rd Semester/DSCI 401/Project/Gold Price.xlsx")
gold['Date'] = pd.to_datetime(gold['Date'])

In [178]:
gold.set_index('Date', inplace=True)
monthly_data = gold['Gold'].resample('M').first()

for month_end in monthly_data.index:
    if pd.isna(monthly_data[month_end]):
        
        start_of_month = month_end.replace(day=1)
        available_data = gold_price_series[start_of_month:month_end]
        if not available_data.empty:
            monthly_data[month_end] = available_data.iloc[0]


gold_filtered = monthly_data.reset_index()
gold_filtered.columns = ['Date', 'Gold']
gold_filtered = gold_filtered[gold_filtered['Date']>'1984-12-31'].reset_index(drop=True)
gold_filtered['Date'] = pd.to_datetime(gold_filtered['Date'])
gold_filtered['Date'] = gold_filtered['Date'].apply(lambda x: x.replace(day=1))

In [179]:
dataframes = [data,gold_filtered]
new_data = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), dataframes)

In [181]:
new_data.head()

,Date,M2V,UNRATE,CPIAUCNS,PPIACO,FEDFUNDS,Adj Close,DCOILWTICO,Gold
0,1985-01-01,1.799,7.3,105.5,103.4,8.35,179.630005,NaN,305.5
1,1985-02-01,1.799,7.2,106.0,103.3,8.50,181.179993,NaN,303.6
2,1985-03-01,1.799,7.2,106.4,103.1,8.58,180.660004,NaN,287.2
3,1985-04-01,1.795,7.3,106.9,103.3,8.27,179.830002,NaN,321.1
4,1985-05-01,1.795,7.2,107.3,103.5,7.97,189.550003,NaN,312.5


## DXY

In [122]:
data = pd.read_csv('F:/Okanagan College/3rd Semester/DSCI 401/Project/Cluster_6.csv')
data['Date'] = pd.to_datetime(data['Date'])

In [123]:
stock_price = yf.download('DX-Y.NYB' , start='1975-01-01',end='2024-11-30',interval="1mo")
stock_price = stock_price.reset_index()
stock_price = stock_price[['Date','Adj Close']]
stock_price['DXY'] = stock_price['Adj Close']
stock_price.drop('Adj Close',axis=1,inplace=True)
stock_price['Date'] = pd.to_datetime(stock_price['Date'])

[*********************100%%**********************]  1 of 1 completed


In [124]:
dataframes = [data,stock_price]
data = reduce(lambda left, right: pd.merge(left, right, on='Date', how='inner'), dataframes)

In [126]:
data.to_csv('Final Data.csv',index=False)